In [35]:
import pandas as pd

In [36]:
df = pd.read_csv("C:/Users/PoojaniRathnayakeBIS/OneDrive - BISTEC Global/Desktop/Music Recomendation_AI/spotify_millsongdata.csv")

In [37]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [38]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [39]:
df.shape

(57650, 4)

In [40]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [41]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [43]:
df.head(10)

,artist,song,text
0,Linda Ronstadt,Lover Man,I don't know why but I'm feeling so sad \r\nI...
1,Robbie Williams,King Of The Bongo,King of the Bongo \r\n(M. Chao) \r\n \r\nMa...
2,Fastball,Emily,Emily shrugs and drags her heels as she takes ...
3,Matt Redman,How Great Is Your Faithfulness,Now unto the King who reigns over all \r\nAnd...
4,Everclear,Drunk Again,Kathy please just go away \r\nDon't let Hazel...
5,Boney M.,Consuela Biaz,In the hills above Freno \r\nBy a shiny mount...
6,Adele,Black And Gold,The fish swam \r\nOut of the ocean \r\nAnd g...
7,Foreigner,Lowdown And Dirty,"This is the thirteenth day of waiting, the sec..."
8,Fun.,Barlights,Never in all of my life \r\nHave I seen eyes ...
9,Christmas Songs,Baby It's Cold Outside,I really can't stay - Baby it's cold outside ...


In [44]:
df.shape

(5000, 3)

In [45]:
df['text'][0]

"I don't know why but I'm feeling so sad  \r\nI long to try something I've never had  \r\nNever had no kissin'  \r\nOoh what I've been missin'  \r\nLover man oh where can you be  \r\n  \r\nThe night is cold and I'm so all alone  \r\nI'd give my soul just to call you my own  \r\nGot a moon above me  \r\nBut there's no one to love me  \r\nLover man oh where can you be  \r\n  \r\nI've heard it said that the thrill of romance can be  \r\nLike a heavenly dream  \r\nI go to bed with a prayer that you'll make love to me  \r\nStrange as it seems  \r\nSomeday we'll meet and you'll dry all my tears  \r\nAnd whisper sweet little things in my ears  \r\nHuggin' and a-kissin'  \r\nOoh what I've been missin'  \r\nLover man oh where can you be  \r\n  \r\nI've heard it said that the thrill of romance can be  \r\nLike a heavenly dream  \r\nI go to bed with a prayer that you'll make love to me  \r\nStrange as it seems  \r\nSomeday we'll meet and you'll dry all my tears  \r\nAnd whisper sweet little thing

In [46]:

df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [47]:
import nltk
from nltk.stem.porter import PorterStemmer

In [48]:
stemmer = PorterStemmer()

In [49]:
def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [26]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\PoojaniRathnayakeBIS\AppData\Roaming\nltk_dat
[nltk_data]     a...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [50]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [51]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [52]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')



In [53]:
matrix = tfidvector.fit_transform(df['text'])


In [54]:
similar = cosine_similarity(matrix)

In [55]:
similar[0]

array([1.        , 0.00267673, 0.04354348, ..., 0.03071302, 0.10413297,
       0.14237537])

In [63]:
df[df['song']=='Black And Gold'].index[0]

6

In [66]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similar[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [67]:
recommendation('Black And Gold')

['Gold',
 'You Got Gold',
 'Heart Of Gold',
 'All That Really Matters',
 "It Doesn't Really Matter",
 'Pressure',
 'Men In Black',
 "3 Chains O' Gold",
 'Wish Upon A Star',
 'The Blues In Black And White',
 "It Just Doesn't Matter",
 'Feel Your Love',
 'Spiralling',
 'Hour Of Gold',
 "Ya'll Don't Hear Me Tho",
 'If You Really Want To Be My Friend',
 'Little Black Sandals',
 'Send Me A Letter From Heaven',
 'My Child',
 'Sweet Gene Vincent']

In [69]:
import pickle
pickle.dump(similar,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))